<a href="https://colab.research.google.com/github/PeiYuJi/tibame/blob/main/tibame_20250430.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import urllib.request as req
import bs4 as bs

def save_post_meta(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())
    # 作者/看版/標題/日期
    metas = html.find_all("span", {"class":"article-meta-value"})
    uid_text = metas[0].text
    board_text = metas[1].text
    title_text = metas[2].text
    time_text = metas[3].text
    # print(uid_text, board_text, title_text, time_text, sep="\n")
    # 每個推虛文 [{"type":"", "id":"", "text":"", "date_ip":""}]
    # JSON格式: {"左者": "看板": "表提": "推虛文":[]}
    push_list = []
    pushes = html.find_all("div", {"class":"push"})
    for push in pushes:
        push_meta = push.find_all("span")
        # 推噓處理
        push_type_text = push_meta[0].text
        if "推" in push_type_text:
            push_type_text = 1
        elif "噓" in push_type_text:
            push_type_text = -1
        else:
            push_type_text = 0
        # 推噓id
        push_id_text = push_meta[1].text
        # 推噓內容
        push_content_text = push_meta[2].text.replace(": ", "")
        # 推噓ipdate
        push_ipdate_text = push_meta[3].text.strip()
        # print(push_type_text, push_id_text, push_content_text, push_ipdate_text, sep="\t")
        push_data = {
            "type":push_type_text,
            "id":push_id_text,
            "text":push_content_text,
            "date_ip":push_ipdate_text
        }
        push_list.append(push_data)
    # print(push_list)
    post = {
        "作者":uid_text,
        "看板":board_text,
        "標題":title_text,
        "日期":time_text,
        "推噓文":push_list,
    }
    # print(post)
    fn = url.split("/")[-1] + ".json"
    with open(fn, "w", encoding="utf-8") as f:
        json.dump(post, f, ensure_ascii=False, indent=4)

當你發現你沒有class/id做篩選的時候
1. 找他老爸
2. 自己想辦法篩選

In [ ]:
import json
import urllib.request as req
import bs4 as bs
import os

def save_post_img(url):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1745582404.A.011.html"
    # prepare directory
    dn = url.split("/")[-1]
    if not os.path.exists(dn):
        os.makedirs(dn)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
    resp = req.urlopen(r)
    html = bs.BeautifulSoup(resp.read())

    allow_sub = ["jpg", "jpeg", "png", "gif"]
    link_list = html.find_all("a")
    for link in link_list:
        link_href = link["href"]
        sub = link_href.split(".")[-1]
        # lower是字串的專屬功能
        if sub.lower() in allow_sub:
            img_url = link_href
            # 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
            r = req.Request(img_url)
            r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
            img_resp = req.urlopen(r)
            img_content = img_resp.read()
            # 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
            # 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
            #                              wb rb -> for 非純文字
            fn = dn + "/" + img_url.split("/")[-1]
            with open(fn, "wb") as f:
                f.write(img_content)

In [ ]:
# os.path這件事我們後面會解釋
import os
# 如果資料夾不存在, 就創起來
if not os.path.exists("a/b"):
    os.makedirs("a/b")

img_url = "https://i.imgur.com/S20cVi5.jpeg"
# 如果對面什麼都沒檢查, 最簡單的一訂是urlretrieve
r = req.Request(img_url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
img_resp = req.urlopen(r)
img_content = img_resp.read()
# 存檔的時候 你要開啟檔案在本地端, 把剛才的內容寫進去
# 之前教你open的時候, 第二個參數 w r  + encoding="utf-8" -> for 純文字
#                              wb rb -> for 非純文字
with open("a/b/test.png", "wb") as f:
    f.write(img_content)

In [ ]:
import json
import urllib.request as req
import bs4 as bs
import os

url = "https://www.ptt.cc/bbs/Beauty/index4000.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36 Edg/135.0.0.0")
resp = req.urlopen(r)
html = bs.BeautifulSoup(resp.read())

div_list = html.find_all("div", {"class":"title"})
for div in div_list:
    post_link = div.find("a")
    if not post_link == None:
        post_link_url = "https://www.ptt.cc" + post_link["href"]
        print(post_link_url)
        save_post_meta(post_link_url)
        save_post_img(post_link_url)


https://www.ptt.cc/bbs/Beauty/M.1748772693.A.772.html
https://www.ptt.cc/bbs/Beauty/M.1748773028.A.FA3.html
https://www.ptt.cc/bbs/Beauty/M.1748777296.A.1A9.html
https://www.ptt.cc/bbs/Beauty/M.1748778750.A.B22.html
https://www.ptt.cc/bbs/Beauty/M.1748778937.A.761.html
https://www.ptt.cc/bbs/Beauty/M.1748779747.A.099.html
https://www.ptt.cc/bbs/Beauty/M.1748780185.A.9D7.html
https://www.ptt.cc/bbs/Beauty/M.1748780278.A.9F9.html
https://www.ptt.cc/bbs/Beauty/M.1748780929.A.004.html
https://www.ptt.cc/bbs/Beauty/M.1748783248.A.F04.html
https://www.ptt.cc/bbs/Beauty/M.1663845030.A.EF9.html
https://www.ptt.cc/bbs/Beauty/M.1621836193.A.468.html
https://www.ptt.cc/bbs/Beauty/M.1666371664.A.B29.html


IndexError: list index out of range

In [ ]:
# 如果你要把雲端硬碟當成隨身碟
dn = "drive/MyDrive/tiba0430"
if not os.path.exists(dn):
    os.makedirs(dn)
fn = dn + "/" + "a.txt"
with open(fn, "w", encoding="utf-8") as f:
    f.write("abcde")

In [ ]:
!zip -r /content/file.zip /content/M.1745582404.A.011.html

	zip warning: name not matched: /content/M.1745582404.A.011.html

zip error: Nothing to do! (try: zip -r /content/file.zip . -i /content/M.1745582404.A.011.html)


In [ ]:
import glob
import json
import pandas as pd
# [{}, {}, {}]
table = []
for fn in glob.glob("M.*.json"):
    with open(fn, "r", encoding="utf-8") as f:
        data = json.loads(f.read())
        table.append(data)

df = pd.json_normalize(table)
df.to_csv("ptt.csv", encoding="utf-8")

0. 中文分析: 分詞 我喜歡你-> 我/喜歡/你
抓出關鍵詞:
1. 出現次數: 越高, 越重要
2. 平常出現的頻率: 越低, 越重要

詞彙重要性(TF-IDF): 次數(TF) * 1/平常出現頻率(IDF)

分詞:  
1. 辭典做初步話分(越大越好)
2. (混淆/不知道) 利用上下文以及機率算法來算出哪種機率高

混淆:乒乓球拍賣掉了

In [ ]:
news = """港星謝霆鋒與張柏芝育有兩子謝振軒（Lucas）及謝振南（Quintus），可惜夫妻倆自2011年與結束5年婚姻。謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，繼日前女友王菲低調捧場之外，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。近日更有港媒報導指出，謝霆鋒的17歲長子Lucas已考入劍橋大學物理系。

謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自謝霆鋒微博、小紅書）
謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自謝霆鋒微博、小紅書）
▲▼謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自謝霆鋒微博、小紅書）

謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自謝霆鋒微博、小紅書）
謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自謝霆鋒微博、小紅書）
謝霆鋒與張柏芝二人一直以來非常重視兩名兒子的成長和教育。大兒子Lucas有別於一般人對「星二代」的刻板印象，學業成績不俗，他與弟弟Quintus就讀位於沙田石門的基督教國際學校（International Christian School，ICS）。據了解，Lucas於15歲時花3年時間便完成5年的課程，而弟弟Quintus則取得全A成績，在學業方面表現相當出色。近日有港媒報導指，長子Lucas考入英國劍橋大學物理系，但父親謝霆鋒和母親張柏芝均未能證實消息。

謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自微博）
謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自微博）
▲謝霆鋒近期在香港啟德體育場舉辦一連4場演唱會，媽媽狄波拉也帶著Lucas、Quintus一起在台下聆聽。（圖／翻攝自微博）

謝霆鋒在演唱會前，下午與兩名兒子到啟德商場內一間餐廳吃飯，被大批網友野生捕獲。（圖／翻攝自微博）
謝霆鋒在演唱會前，下午與兩名兒子到啟德商場內一間餐廳吃飯，被大批網友野生捕獲。（圖／翻攝自微博）
▲謝霆鋒在演唱會前，下午與兩名兒子到啟德商場內一間餐廳吃飯，被大批網友野生捕獲。（圖／翻攝自微博）

謝霆鋒在27日演唱會尾場開唱前，下午與兩名兒子到啟德商場內一間餐廳吃飯，被大批網友野生捕獲。可見謝霆鋒與兩名兒子先後進入食店，現場有客人即時認出三人。兩名兒子坐一排，謝霆鋒則坐在二人對面，謝霆鋒與兩子說話，而兩名兒子亦有回應父親，父子三人言談甚歡和諧。而當三人用餐完畢後便離開，謝霆鋒行最前，隨後的是Quintus及Lucas，Lucas再次面露微笑與現場熱情的粉絲打招呼，獲網友大讚親切有禮。"""


In [ ]:
import jieba
import urllib.request as req
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
req.urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
" ".join(jieba.cut(news))

In [ ]:
import jieba.analyse

jieba.analyse.extract_tags(news)